In [1]:
from cortexlib.utils.plotting import simclr_colours, vgg19_colours, gabor_filter_colours
from cortexlib.utils import file as futils
from cortexlib.utils.logging import Logger
import pandas as pd
import matplotlib.pyplot as plt

logger = Logger()

In [2]:
MOUSE_ID = futils.get_mouse_id()
logger.info(f"This notebook is running for mouse {MOUSE_ID}")

15:41:59 | INFO     | ℹ️ This notebook is running for mouse m01_d2


In [3]:
simclr_fev_results = futils.read_json_file_as_dataframe('../04_simclr_neural_pc1/prediction.json')
best_fev_per_simclr_layer = simclr_fev_results.loc[simclr_fev_results.groupby("layer")["mean_fev"].idxmax()]

best_fev_per_simclr_layer

,layer,n_pcs,test_r2,mean_fev
43,fc,400.0,0.492836,0.520464
2,layer1,20.0,0.442110,0.467287
12,layer2,50.0,0.497531,0.525386
18,layer3,NaN,0.518632,0.547507
27,layer4,NaN,0.482073,0.509181


In [4]:
vgg19_fev_results = futils.read_json_file_as_dataframe('../06_vgg19_neural_pc1/prediction.json')
best_fev_per_vgg19_layer = vgg19_fev_results.loc[vgg19_fev_results.groupby("layer")["mean_fev"].idxmax()]

best_fev_per_vgg19_layer

,layer,n_pcs,test_r2,mean_fev
0,conv2_2,NaN,0.407432,0.430933
5,conv3_4,NaN,0.416551,0.440492
10,conv4_4,NaN,0.396909,0.419901
15,conv5_4,NaN,0.281060,0.298454
20,fc2,NaN,0.266597,0.283292


In [5]:
gabor_fev_results = futils.read_json_file_as_dataframe('../02_gabor_neural_pc1/prediction.json')
best_fev_per_gabor_filter_bank = gabor_fev_results.loc[gabor_fev_results.groupby("filter")["mean_fev"].idxmax()]

best_fev_per_gabor_filter_bank

,filter,n_pcs,test_r2,mean_fev
15,gabor_large,5.0,-0.003620,0.000017
9,gabor_medium,5.0,-0.003626,0.000011
3,gabor_small,5.0,-0.003624,0.000013


In [6]:
# Add model identifiers
best_fev_per_simclr_layer['model'] = 'SimCLR'
best_fev_per_vgg19_layer['model'] = 'VGG19'
best_fev_per_gabor_filter_bank['model'] = 'Gabor'

# Rename layer/filter column to a common name
best_fev_per_simclr_layer = best_fev_per_simclr_layer.rename(columns={"layer": "representation"})
best_fev_per_vgg19_layer = best_fev_per_vgg19_layer.rename(columns={"layer": "representation"})
best_fev_per_gabor_filter_bank = best_fev_per_gabor_filter_bank.rename(columns={"filter": "representation"})

# Combine all into a single DataFrame
combined = pd.concat([
    best_fev_per_simclr_layer,
    best_fev_per_vgg19_layer,
    best_fev_per_gabor_filter_bank
])

combined

,representation,n_pcs,test_r2,mean_fev,model
43,fc,400.0,0.492836,0.520464,SimCLR
2,layer1,20.0,0.442110,0.467287,SimCLR
12,layer2,50.0,0.497531,0.525386,SimCLR
18,layer3,NaN,0.518632,0.547507,SimCLR
27,layer4,NaN,0.482073,0.509181,SimCLR
0,conv2_2,NaN,0.407432,0.430933,VGG19
5,conv3_4,NaN,0.416551,0.440492,VGG19
10,conv4_4,NaN,0.396909,0.419901,VGG19
15,conv5_4,NaN,0.281060,0.298454,VGG19
20,fc2,NaN,0.266597,0.283292,VGG19


In [7]:
# Create a unique label for plotting
combined['label'] = combined['model'] + ' - ' + combined['representation']

colour = {
    'SimCLR': simclr_colours['layer3'],
    'VGG19': vgg19_colours['conv4_4'],
    'Gabor': gabor_filter_colours['gabor_medium']
}
colours = combined['model'].map(colour)

plt.figure(figsize=(12, 5))
plt.bar(combined['label'], combined['mean_fev'], color=colours)
plt.ylabel('Mean FEV')
plt.title('Best Mean FEV per Model Representation (PC1 Neural Data)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()